In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *


### **Data reading**

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@databricksetecharif.dfs.core.windows.net/products")
df.display()

In [0]:
df =df.drop("_rescued_data")

In [0]:
df.display()

In [0]:
df.createOrReplaceTempView("products")

In [0]:
%sql
select * from products

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_catalog.bronze.discount(price DOUBLE)
RETURNS DOUBLE
LANGUAGE SQL
RETURN price * 0.90

In [0]:
%sql
SELECT  product_name, int(price) AS original_price, int(databricks_catalog.bronze.discount(price)) AS price_after_discount
from products


In [0]:
df1 = df

In [0]:
df1.display()

In [0]:
df2 = df1.withColumn("discount_price",expr("databricks_catalog.bronze.discount(price)"))

In [0]:
df1.display()

In [0]:
df2.display()

In [0]:
%sql
select * from products

In [0]:
%sql
SELECT product_name, upper(brand) as marka
from products

In [0]:
df1.display()

In [0]:
df = df1

In [0]:
df.display()

In [0]:
df.write.format("delta").mode("append").save("abfss://silver@databricksetecharif.dfs.core.windows.net/products")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_catalog.silver.products_silver
USING DELTA
LOCATION 'abfss://silver@databricksetecharif.dfs.core.windows.net/products'

In [0]:
%sql
select * from databricks_catalog.silver.products_silver